In [1]:
import keras
import pickle
import numpy as np
import os
from keras.models import Model
from keras.layers import Input, Dense, Reshape, dot
from keras.layers.embeddings import Embedding

Using TensorFlow backend.
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
names=["vocab_size","labels","word_target","word_context","window_size"]
for name in names:
    with open(os.path.join(os.path.join(os.path.abspath('../binaries'),"pickles"),name+".pickle"), "rb") as f:
        globals()[name]=pickle.load(f)

In [3]:
vector_dim=300
epochs=1000000

In [4]:
input_target = Input((1,))
input_context = Input((1,))

W1129 10:33:38.428618 24760 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1129 10:33:38.449561 24760 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [5]:
embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

W1129 10:33:38.755780 24760 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
# setup a cosine similarity operation which will be output in a secondary model
similarity = dot([target,context],axes=1,normalize=True)

In [7]:
dot_product = dot([target, context], normalize=False, axes=1)
dot_product = Reshape((1,))(dot_product)
output = Dense(1, activation='sigmoid')(dot_product)
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  after removing the cwd from sys.path.
W1129 10:33:39.089839 24760 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1129 10:33:39.101778 24760 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1129 10:33:39.104770 24760 deprecation.py:323] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will

In [8]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 1000 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
        model.save(os.path.join(os.path.join(os.path.abspath('../binaries'),"models"),"classification_model.h5"))


Iteration 538000, loss=1.192093321833454e-07
Iteration 539000, loss=1.192093321833454e-07
Iteration 540000, loss=0.49969592690467834
Iteration 541000, loss=1.5497212189075071e-06
Iteration 542000, loss=0.06890684366226196
Iteration 543000, loss=0.13895699381828308
Iteration 544000, loss=0.19937723875045776
Iteration 545000, loss=1.3841215372085571
Iteration 546000, loss=2.4229722023010254
Iteration 547000, loss=1.192093321833454e-07
Iteration 548000, loss=0.014032488688826561
Iteration 549000, loss=0.060790758579969406
Iteration 550000, loss=1.192093321833454e-07
Iteration 551000, loss=0.11824224144220352
Iteration 552000, loss=1.192093321833454e-07
Iteration 553000, loss=0.3683505356311798
Iteration 554000, loss=1.192093321833454e-07
Iteration 555000, loss=0.011266155168414116
Iteration 556000, loss=1.2042778730392456
Iteration 557000, loss=0.1248830035328865
Iteration 558000, loss=0.18558184802532196
Iteration 559000, loss=0.1895647495985031
Iteration 560000, loss=0.0001182626510853

In [9]:
model.save(os.path.join(os.path.join(os.path.abspath('../binaries'),"models"),"classification_model.h5"))